In [ ]:
import xml.etree.ElementTree as ET

def parse_sbml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    ns_map = root.attrib  # Namespace map
    sbml_ns = ns_map.get('xmlns', 'http://www.sbml.org/sbml/level3/version1/core')
    qual_ns = ns_map.get('xmlns:qual', 'http://www.sbml.org/sbml/level3/version1/qual/version1')
    math_ns = ns_map.get('xmlns:mathml', 'http://www.w3.org/1998/Math/MathML')
    
    ns = {'sbml': sbml_ns, 'qual': qual_ns, 'math': math_ns}

    # Initialize a dictionary to store the rules for each node
    node_rules = {}

    # Extract rules from transitions
    transitions = root.findall('.//qual:transition', ns)
    for transition in transitions:
        output_species = transition.find('.//qual:output', ns).attrib.get(f'{{{qual_ns}}}qualitativeSpecies')
        function_term = transition.find('.//qual:functionTerm', ns)
        if function_term is not None:
            math_element = function_term.find('.//math:math', ns)
            if math_element is not None:
                # Convert the math element to a string representation of the rule
                rule_str = ET.tostring(math_element, encoding='unicode', method='text')
                node_rules[output_species] = rule_str.strip()
    
    return node_rules

nodes_model1_rules = parse_sbml('Ikonomi2020_updated.sbml')
nodes_model2_rules = parse_sbml('Palma2021.sbml')

# Print out the results to verify
print("Rules in Model 1:", nodes_model1_rules)
print("Rules in Model 2:", nodes_model2_rules)


In [ ]:
# Read the CSV file with custom rules for overlapping nodes
import pandas as pd
rules_df = pd.read_csv('rules.csv')
csv_rules_dict = rules_df.set_index('node')['rules'].to_dict()

# Merge the nodes from both models
merged_nodes_rules = {}

# Add all nodes and rules from the first model
merged_nodes_rules.update(nodes_model1_rules)

# Merge nodes and rules from the second model
for node_id, rule in nodes_model2_rules.items():
    if node_id in merged_nodes_rules:
        # If the node is in both models, use the rule from the CSV if available
        if node_id in csv_rules_dict:
            merged_nodes_rules[node_id] = csv_rules_dict[node_id]
    else:
        # If the node is only in the second model, add it directly
        merged_nodes_rules[node_id] = rule

# Print out the results to verify
print("Merged Nodes with Rules:", merged_nodes_rules)

In [ ]:
# Create the root element of the SBML document
sbml_ns = "http://www.sbml.org/sbml/level3/version1/core"
qual_ns = "http://www.sbml.org/sbml/level3/version1/qual/version1"
ns = {'sbml': sbml_ns, 'qual': qual_ns}
attrib = {
    'xmlns': sbml_ns,
    'level': '3',
    'version': '1',
    'xmlns:qual': qual_ns
}
sbml_root = ET.Element(f'{{{sbml_ns}}}sbml', attrib)

# Create the model element
model = ET.SubElement(sbml_root, f'{{{sbml_ns}}}model', {'id': 'merged_model'})

# Create the listOfQualitativeSpecies element within the model
list_of_qual_species = ET.SubElement(model, f'{{{qual_ns}}}listOfQualitativeSpecies')

# Create the listOfTransitions element within the model
list_of_transitions = ET.SubElement(model, f'{{{qual_ns}}}listOfTransitions')

for node_id, rule in merged_nodes_rules.items():
    # Add qualitativeSpecies to listOfQualitativeSpecies
    qual_species = ET.SubElement(list_of_qual_species, f'{{{qual_ns}}}qualitativeSpecies', {
        'id': node_id,
        'compartment': 'default',  # Update with the actual compartment if necessary
        'constant': 'false',
        'maxLevel': '1'
    })
    
    # Add transition for this species to listOfTransitions
    transition = ET.SubElement(list_of_transitions, f'{{{qual_ns}}}transition')
    
    # Add listOfInputs (empty, as we're directly using rules)
    ET.SubElement(transition, f'{{{qual_ns}}}listOfInputs')
    
    # Add listOfOutputs
    list_of_outputs = ET.SubElement(transition, f'{{{qual_ns}}}listOfOutputs')
    ET.SubElement(list_of_outputs, f'{{{qual_ns}}}output', {
        'qualitativeSpecies': node_id,
        'transitionEffect': 'assignmentLevel'
    })
    
    # Add listOfFunctionTerms
    list_of_function_terms = ET.SubElement(transition, f'{{{qual_ns}}}listOfFunctionTerms')
    
    # Add a defaultTerm (assuming default level is 0)
    ET.SubElement(list_of_function_terms, f'{{{qual_ns}}}defaultTerm', {'resultLevel': '0'})
    
    # Add functionTerm with the rule
    function_term = ET.SubElement(list_of_function_terms, f'{{{qual_ns}}}functionTerm', {'resultLevel': '1'})
    
    # Add math (rule) to functionTerm
    math = ET.SubElement(function_term, f'{{{qual_ns}}}math', {'xmlns': 'http://www.w3.org/1998/Math/MathML'})
    # Here you can parse the 'rule' and add the corresponding elements to 'math'
    # This step might be complex as it requires parsing the rule's text and converting it to MathML
    # As a placeholder, let's just add the rule as a comment
    math.text = rule  # This is a simplification, actual MathML construction is needed

tree = ET.ElementTree(sbml_root)
tree.write('merged.sbml', encoding='UTF-8', xml_declaration=True)


In [ ]:
merged = biolqm.load("merged.sbml")
merged_lrg = biolqm.to_ginsim(merged)
ginsim.show(merged_lrg)